# Importing Libraries and opening ms edge 
### This program will create a directory called 'output' and is where it stores scraped tweets.

In [18]:
import os
if not os.path.exists("./output"):
    os.makedirs('output')
from textblob import TextBlob
from msedge.selenium_tools import Edge, EdgeOptions
import matplotlib.pyplot as plt
import numpy as np
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options = options)
import time
import csv
import datetime
import nltk
import string
import re

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('brown')
nltk.download('punkt')
stopword = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\codyw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\codyw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\codyw\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\codyw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data cleaning functions
## Includes:
###                    * remove_stopwords(string text):
####                                  -removes stop words from a provided string (stop words are words in a language that are spoke the most)
###                    * remove_punct(string text):
####                                  -removes punctuation from provided string.
###                    * tokenization(text):
####                                  -splits the string in to tokens

In [19]:
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub(r'[0-9]+', '', text)
    return text

def tokenization(text):
    text = re.split('\W+', text)
    return text


##### Navigate ms edge and setting up the query. 
##### Note: the reason why these code blocks are so small is to allow rapid rerunning given a spotty internet connection

In [20]:
driver.get("Https://twitter.com/explore")
time.sleep(3)

##### Locate the query bar and send it a query
##### Note: You have the option to scrape two different types of test data: 
##### 1) searches for I  (it is one of the most common neutral words out there giving us a true random sample) 
##### 2) search for recent tweets that contain one of the key words  (default)                                                                     


In [21]:
search_input = driver.find_element_by_xpath('//input[@aria-label = "Search query"]')

#comment me out for option 1 (uncomment option 2)
search_input.send_keys('depression OR depressed OR therapy OR antidepressant OR prozac OR xanax lang:en -filter:replies')

#comment me out for option 2 (uncomment option 1)
#search_input.send_keys('I lang:en -filter:replies')

search_input.send_keys(Keys.RETURN)
time.sleep(3)

###### Sort by latest.

In [22]:
driver.find_element_by_link_text('Latest').click()
time.sleep(3)

##### Scrape handles

In [24]:
tweets = driver.find_elements_by_xpath('//div[@data-testid = "tweet"]')
tweet = tweets[0]
print("Number of handles pulled:")
print(len(tweets))


print("Aquired handles")

handles = []

for lookup in tweets:
    handles.append(lookup.find_element_by_xpath('.//span[contains(text(), "@")]').text)
for h in handles: 
    print(h)

Number of handles pulled:
25
Aquired handles
@koncerndd
@myselfnil
@TheGreenSerpent
@Abhishe82333926
@Afailureofamess
@DepressedHonk
@FezInRealLife
@__catholicguilt
@waifudarling
@SubFoxQueen
@darrenhayman
@PaulBro01586772
@Mito_Therapy
@BillyRalph
@phethani4
@WJS_Nyangulu
@Limbictweets
@Imwolfyy_
@tarlos118
@MiraaAree
@kerma4kHD
@yahunnaki
@depression_cow
@taayurveda
@arab1000com


##### Setting up keyword list (affects keyword score) and iList for (1stperson_score)
##### Note to grader: You can change maxTweets if runtime gets too long to lower the amount of tweets you pull from each handle

In [25]:
maxTweets = 20
#handles = ["@ChingChingChooi","@buardian","@tooheadache","@shodorokik", "@kyleKitchen5", "@shit_mirror", "@garbagobean", "@helloitsaudrey", "@relatable_3am", "@ollopaahtnamas", "@Mo_thepoet", "@gladiola666"]
keylist= ["depression", "depressed", "suicide", "therapy", "therapist", "xanax", "antidepressant", "prozac"]
ilist= ["I", "Iam", "I'm", "I am", "Me", "My", "i", "iam", "i'm", 'im']

## Scrape the list of handle's Twitter history
###### This pulls around 20 tweets per "pass". We use the date and some twitter query know how to exclude previously scraped tweets.
###### On each tweet, Date is recorded(for query and our days inbetween attribute) and tweet is recorded
###### The tweet is then cleaned by removing punctuation, tokenizing, removing the stopwords and returned to a string.
###### Both the uncleaned and cleaned tweets are sent to textblob where both are scored by polarity and subjectivity
###### Further more, textblob will attempt to find noun phrases in the tweet that we use to search for 1st person nouns.
###### The tweet string is parsed for keywords and scored by unique appreances.
###### Before moving to a new file the scraper will write two .csvs for the finished handle one is for reading to help understanding methidology, the other is the data fed to the NN

In [26]:
import string
printfilter = set(string.printable)
messages=[]   #this will save handles into a csv
numericMessages=[] #this will only save numerical data to send to the NN
numericMessages.append(list(("label", "polar" , "subjective", "year", "month", "day","in_between","keyword_score", "1stperson_score")))
messages.append(list(("handle", "Tweet","polar" , "subjective", "year", "month", "day","in between","keyword score", "1st person noun phrase score","Polarity (cleaned)", "Subjectivity (cleaned)", "keyword score (cleaned)", "1st person noun phrase score(cleaned)" )))
bnotfirst = False; #used for days in between
idate = ""
returnedtweets = 0
numtweets = 0
import csv





search_input = driver.find_element_by_xpath('//input[@aria-label = "Search query"]')

for handle in handles:
    while numtweets<maxTweets:
        for i in range(0,200):
            search_input.send_keys(Keys.BACKSPACE)
        if numtweets == 0:    
            search_input.send_keys('from:'+handle+' lang:en -filter:replies')  #searches for handles from the inital depression search
        else:
            search_input.send_keys('from:'+handle+' lang:en -filter:replies until:'+idate) #searches for tweets from handles utilizing the date to scrape more
        search_input.send_keys(Keys.RETURN)
        time.sleep(3)        
        tweets = driver.find_elements_by_xpath('//div[@data-testid = "tweet"]') #pulls a unknown number of tweets
        
        numtweets = numtweets+ len(tweets)
        if len(tweets) == 0:
            break
        for tweet in tweets:
            message = tweet.find_element_by_xpath('.//div[2]/div[2]/div[1]').text #pulls the message
            messagept = tweet.find_element_by_xpath('.//div[2]/div[2]/div[2]').text #pulls the message
            
            idate = tweet.find_element_by_xpath('.//time').get_attribute('datetime') #pulls the date
            
            month = int(idate.split('-')[1])
            year =  int(idate.split('-')[0])
            sday =  (idate.split('-')[2]) 
            day = int(sday.split('T')[0]) 
            currentDay = datetime.datetime(year, month, day) #date processing
            if bnotfirst == False:
                bnotfirst=True
                inbetween = 0
            else:
                inbetween = previousDay - currentDay
                inbetween = inbetween.days
            previousDay = currentDay
            
            inputmessage = message + messagept #combines the the two halfs of the tweet
            print("uncleaned", inputmessage)
            inputmessage = ''.join(filter(lambda x: x in printfilter, inputmessage))
            
            sentAnalysis = TextBlob(inputmessage) #uncleaned to textblob
            nounAnalysis = sentAnalysis.noun_phrases
            
            inputmessage.replace("\n", " ")
            
            #cleaning data
            inputmessage = remove_punct(inputmessage) #removing punctuation
            
            
            tokenmessage = tokenization(inputmessage)  #tokenizing        
            
            
            stopwords = remove_stopwords(tokenmessage) #removing stopwords
            
            cleanedString = " "
            cleanedString = cleanedString.join(stopwords) #reform the string
            print("cleaned", cleanedString)
            cleanAnalysis = TextBlob(cleanedString) #send back into text blob
            cleanednounAnalysis = cleanAnalysis.noun_phrases
         
            
            cleanpolar = cleanAnalysis.sentiment.polarity #run textblob on cleaned and uncleaned data
            polar = sentAnalysis.sentiment.polarity
            
            cleansubjective = cleanAnalysis.sentiment.subjectivity
            subjective = sentAnalysis.sentiment.subjectivity
                     
            matching = [s for s in keylist if s in inputmessage] #sperate looking ofr keywords to help network (looking for depression based keywords)
            cmatching = [s for s in keylist if s in cleanedString] #same for hte cleaned data
            
            cnMatching = [s for s in cleanednounAnalysis if any(n in s for n in ilist)] # looking for first person noun phrases
            nMatching = [s for s in nounAnalysis if any(n in s for n in ilist)] 
            
            cleanedkeywordscore = len(cmatching) #the number of instances of keyword matches
            keywordscore = len(matching)
            
            cleanednscore = len(cnMatching)
            nscore = len(nMatching)
                
            messages.append(list((handle, inputmessage,polar , subjective, year, month, day,inbetween,keywordscore, nscore, cleanpolar, cleansubjective, cleanedkeywordscore, cleanednscore)))
            numericMessages.append(list(("0", cleanpolar, cleansubjective, year, month, day,inbetween,cleanedkeywordscore,cleanednscore )))
        if len(tweets)==0:
            break
    
    messages.append(list(("","","")))
    numericMessages.append(list(("","","")))
    
    file = "./output/"+handle+ ".csv"
    with open(file, 'a', newline='') as csvfile:
        tweetwriter = csv.writer(csvfile, delimiter=',', quoting = csv.QUOTE_MINIMAL)
        for writemessage in messages:
            tweetwriter.writerow(writemessage)
        
    numfile="./output/number" + handle + ".csv"
    with open(numfile, 'a', newline='') as csvfile:
        numwriter = csv.writer(csvfile, delimiter=',', quoting = csv.QUOTE_MINIMAL)
        for writemessage in numericMessages:
            numwriter.writerow(writemessage)

    
    messages = []
    numMessages = []
    polarityAvg = 0
    inbetween = 0
    bnotfirst = False
    idate = ""
    returnedtweets = 0
    numtweets = 0
   

messages

uncleaned life sucks when anxiety has you constantly worried but depression makes you too tired to really care
cleaned life sucks anxiety constantly worried depression makes tired really care
uncleaned fanboy and chum chum is still the most underrated nickelodeon show idc
cleaned fanboy chum chum still underrated nickelodeon show idc
uncleaned the energy they feed you is not how they really feel remember that
cleaned energy feed really feel remember
uncleaned yall clowning nle choppa for no reason..yk damn well youll get dropped by that nigga
cleaned yall clowning nle choppa reasonyk damn well youll get dropped nigga
uncleaned i hate n*ggas who listen to leaks 
cleaned hate nggas listen leaks 
uncleaned i lowkey wanna have s*x with 
@Klo4k
cleaned lowkey wanna sx Klok
uncleaned new 
@happyhxliday
 songSalsa - TheHxliday
open.spotify.com
cleaned new happyhxliday songSalsa TheHxliday openspotifycom
uncleaned overthinking may killed my energy, but i was never wrong 
cleaned overthinking m

uncleaned Make mr singh speaker and I'll sleep ffs
cleaned Make mr singh speaker Ill sleep ffs
uncleaned "host won't speak" 
cleaned host wont speak 
uncleaned My randi rona hrs have been underutilized in past few days, and Hari Om to my throat 
cleaned My randi rona hrs underutilized past days Hari Om throat 
uncleaned This song
cleaned This song
uncleaned Come out kumar sanu, we've competition 
cleaned Come kumar sanu weve competition 
uncleaned Mc firr
cleaned Mc firr
uncleaned World is not ready for Pratyush madness
cleaned World ready Pratyush madness
uncleaned Want more thrillers like 
Don't breathe
Orphan
Get out  
cleaned Want thrillers like Dont breathe Orphan Get 
uncleaned "The Babysitter" with blank mindQuote Tweet
Netflix India
@NetflixIndia
 · May 4
A show/movie you think everyone should watch 
cleaned The Babysitter blank mindQuote Tweet Netflix India NetflixIndia May A showmovie think everyone watch 
uncleaned Things that kept me alive in tough times, why do I not find 

uncleaned ||Wanna draw Jackie in this one day
cleaned Wanna draw Jackie one day
uncleaned ||So much for me trying to relax today. Seriously just fuck my family.
cleaned So much trying relax today Seriously fuck family
uncleaned ||I am gonna nap
cleaned I gonna nap
uncleaned "Him and Bruce aren't really seeing eye to eye right now."
cleaned Him Bruce arent really seeing eye eye right
uncleaned "Says me. Come on Fangs. He has no where to sleep tonight."
cleaned Says Come Fangs He sleep tonight
uncleaned "So Barry is gonna be with us for a bit."
cleaned So Barry gonna us bit
uncleaned "This robe makes breast feeding my boys easy."
cleaned This robe makes breast feeding boys easy
uncleaned "If my boys didn't need theater songs to learn I would've hurt you for that."
cleaned If boys didnt need theater songs learn I wouldve hurt
uncleaned "My boys are gonna be singing Hamilton in the future. Aren't they?"
cleaned My boys gonna singing Hamilton future Arent
uncleaned "Perfect might turn into 

uncleaned The moment I see an artist supporting any form of NFTs for any reason, I'm unfollowing. I don't care why someone is making NFTs. I will not support anyone that is for supporting scams.
cleaned The moment I see artist supporting form NFTs reason Im unfollowing I dont care someone making NFTs I support anyone supporting scams
uncleaned Great, another event in Genshin that I need friends to do it with. Guess I won't be enjoying this event.
cleaned Great another event Genshin I need friends Guess I wont enjoying event
uncleaned wtf is this event going on in Genshin? I'm so confused. It doesn't explain a damn thing.
cleaned wtf event going Genshin Im confused It doesnt explain damn thing
uncleaned It breaks my heart to see so many people hope that Kaeya and Zhongli are evil. T^T They're good boys.
cleaned It breaks heart see many people hope Kaeya Zhongli evil TT Theyre good boys
uncleaned Omg yes her face is some what fixed. Thank you. Quote Tweet
Dead by Daylight
@DeadByBHVR
 · 

uncleaned my song was number 15 on the unearthed charts and I didn’t even know
cleaned song number unearthed charts I didnt even know
uncleaned appreciation tweet for my manager who has not yet said no to any of my dumb ideas i love u 
@beatnikratbag
cleaned appreciation tweet manager yet said dumb ideas love u beatnikratbag
uncleaned how do i start selling songs to other artists and producers? i got 800 pop hooks and no use for them i wanna make some money
cleaned start selling songs artists producers got pop hooks use wanna make money
uncleaned i took selfies i don’t hate lmaoThe following media includes potentially sensitive content. Change settings
View
cleaned took selfies dont hate lmaoThe following media includes potentially sensitive content Change settings View
uncleaned being a child and having adult women tell u that ur “gonna have to fix that nose when you’re older” will really play on ur self esteem forever
cleaned child adult women tell u ur gonna fix nose youre older rea

...

uncleaned Did you know the number one thing that kills fiddle leaf figs are the gays?
cleaned Did know number one thing kills fiddle leaf figs gays
uncleaned To the gays; you can bleach your hole but you can’t put on a mask??
cleaned To gays bleach hole cant put mask
uncleaned Got my second vaccine shot and now I see through walls and my grew 3 feet so there’s no excuse to not get it when you can
cleaned Got second vaccine shot I see walls grew feet theres excuse get
uncleaned A still life
cleaned A still life
uncleaned My coworker is willingly missing his second Covid vaccine shot tomorrow... because he’s an idiot and I hope he chokes 
cleaned My coworker willingly missing second Covid vaccine shot tomorrow hes idiot I hope chokes 
uncleaned Starbucks really came through today giving me my will to live
cleaned Starbucks really came today giving live
uncleaned I hate twinks who don’t know how to take care of plants
cleaned I hate twinks dont know take care plants
uncleaned Some of y’al

uncleaned “𝗔𝘆𝘂𝗿𝘃𝗲𝗱𝗶𝗰 Detoxification ”𝗮𝘁 "𝗧𝗿𝗮𝘃𝗮𝗻𝗰𝗼𝗿𝗲 𝗔𝘆𝘂𝗿𝘃𝗲𝗱𝗮 "
𝗗𝗲𝘁𝗼𝘅 - 𝗳𝗼𝗿 𝗮 𝗵𝗮𝗽𝗽𝗶𝗲𝗿, 𝗵𝗲𝗮𝗹𝘁𝗵𝗶𝗲𝗿 𝗮𝗻𝗱 𝗲𝗻𝗲𝗿𝗴𝗲𝘁𝗶𝗰 𝗹𝗶𝗳𝗲
https://travancoreayurveda.com/detoxification/
cleaned  Detoxification httpstravancoreayurvedacomdetoxification
uncleaned Travancore Ayurveda Kanchanara Guggulu available now on one of the most popular e -commerce platform Flipkart
click below link for more detailsFlipkart - Online Shopping App
Shop for electronics, apparels & more using our Flipkart app Free shipping & COD.
4.56219/5.0 stars – 1,830,875 ratings
cleaned Travancore Ayurveda Kanchanara Guggulu available one popular e commerce platform Flipkart click link detailsFlipkart Online Shopping App Shop electronics apparels using Flipkart app Free shipping COD stars ratings
uncleaned 𝘼𝙮𝙪𝙧𝙫𝙚𝙙𝙖 is a system of medicine with historical roots in the Indian subcontinent. Globalized and modernized practices derived from Ayurveda traditions are a type of alternative medicine
cleaned  system medicine historical roots Indian s

uncleaned Larry The Cable Guy Food Taste Test -Larry The Cable Guy Food Taste Test
We’re putting Larry the Cable Guy’s various food products to the test to see which one can really GIT R DONE. GMM #1217 Watch Part 2: | Watch yesterday’s GMM: Watch today’s …
arab1000.com
cleaned Larry The Cable Guy Food Taste Test Larry The Cable Guy Food Taste Test Were putting Larry Cable Guys various food products test see one really GIT R DONE GMM Watch Part Watch yesterdays GMM Watch todays arabcom
uncleaned Attracting An Alpaca With An Accordion | Nowhere Else -Attracting An Alpaca With An Accordion | Nowhere Else
Trust us, there’s nowhere else in the universe that you’ll see something like this. GMM #1217 Watch GMMore: | Part 3: Watch today’s episode from the start: Pick up all of the offi…
arab1000.com
cleaned Attracting An Alpaca With An Accordion Nowhere Else Attracting An Alpaca With An Accordion Nowhere Else Trust us theres nowhere else universe youll see something like GMM Watch GMMore Part

uncleaned Assassins Creed Odyssey: Dumb Yet HILARIOUS Glitches -Assassins Creed Odyssey: Dumb Yet HILARIOUS Glitches
Assassin’s Creed Odyssey (PC, PS4, Xbox One) is a fun game, but it’s not without a few funny glitches and bugs. Here are the best ones that YOU sent us! Subscribe for more:
arab1000.com
cleaned Assassins Creed Odyssey Dumb Yet HILARIOUS Glitches Assassins Creed Odyssey Dumb Yet HILARIOUS Glitches Assassins Creed Odyssey PC PS Xbox One fun game without funny glitches bugs Here best ones YOU sent us Subscribe arabcom
uncleaned Xbox Game Pass vs PlayStation NOW -Xbox Game Pass vs PlayStation NOW
Video game services on consoles are on the rise. Sony and Microsoft are competing in new and innovative ways – let’s take a deeper look! Subscribe for more:
arab1000.com
cleaned Xbox Game Pass vs PlayStation NOW Xbox Game Pass vs PlayStation NOW Video game services consoles rise Sony Microsoft competing new innovative ways lets take deeper look Subscribe arabcom
uncleaned PS5 'NECES

[]